# Lesson 2: First Tool - Extending Agent Capabilities

This interactive notebook introduces **tools** - the foundation of agent capabilities:

- ✅ The `@tool` decorator for function-based tools
- ✅ Tool specifications and JSON schemas
- ✅ Tool invocation logic by the agent
- ✅ Tool result handling and formatting
- ✅ Error handling in tools (try/except patterns)
- ✅ Tool descriptions for proper selection

**Estimated time:** 2 hours

**What you'll build:** An agent with calculator capabilities that can perform complex math!

## Setup

Import necessary modules and configure the environment:

In [ ]:
import asyncio
from lesson_utils import (
    load_environment,
    create_working_model,
    check_api_keys,
    print_troubleshooting
)
from strands import Agent, tool

# Load environment and check API keys
load_environment()
check_api_keys()

print("🎯 Lesson 2: First Tool - Calculator")
print("=" * 40)

## Part 1: Creating Your First Tool

The `@tool` decorator transforms a regular Python function into an agent tool. The agent will:
1. Read the function's docstring to understand what it does
2. See the type hints to know what parameters to pass
3. Call the function when needed and use the result

Let's create a calculator tool:

In [ ]:
@tool
def calculator(expression: str) -> str:
    """
    Evaluate mathematical expressions safely.

    This tool can handle basic arithmetic operations including:
    - Addition (+), Subtraction (-), Multiplication (*), Division (/)
    - Exponentiation (**)
    - Parentheses for grouping

    Args:
        expression: A mathematical expression as a string (e.g., "2 + 3 * 4")

    Returns:
        str: The result of the calculation or an error message

    Examples:
        calculator("2 + 3") → "5"
        calculator("10 / 2") → "5.0"
        calculator("2 ** 3") → "8"
    """
    try:
        import math
        import builtins

        # Get safe built-in functions
        safe_builtins = ['abs', 'round', 'min', 'max', 'pow']
        allowed_names = {}
        for name in safe_builtins:
            if hasattr(builtins, name):
                allowed_names[name] = getattr(builtins, name)

        # Add math constants and functions
        allowed_names.update({
            'pi': math.pi,
            'e': math.e,
            'sqrt': math.sqrt,
            'sin': math.sin,
            'cos': math.cos,
            'tan': math.tan,
        })

        # Evaluate the expression safely
        result = eval(expression, {"__builtins__": {}}, allowed_names)

        # Format the result nicely
        if isinstance(result, float) and result.is_integer():
            return str(int(result))
        return str(result)

    except ZeroDivisionError:
        return "Error: Division by zero is not allowed"
    except SyntaxError:
        return f"Error: Invalid mathematical expression '{expression}'"
    except NameError as e:
        return f"Error: Unknown function or variable in '{expression}'"
    except Exception as e:
        return f"Error: Could not evaluate '{expression}' - {str(e)}"

print("✅ Calculator tool created!")

## Part 2: Agent with Calculator Tool

Now let's create an agent and give it the calculator tool. The agent will automatically know when to use it!

In [ ]:
model = create_working_model()

if model:
    # Create agent with the calculator tool
    agent = Agent(
        model=model,
        tools=[calculator],  # Register our calculator tool
        system_prompt="""You are a helpful assistant with calculator capabilities.
        When someone asks a math question, use the calculator tool to get accurate results.
        Always explain your calculations clearly."""
    )

    # Test basic calculations
    test_questions = [
        "What is 15 + 27?",
        "Calculate 144 divided by 12",
        "What's 2 to the power of 8?",
        "Can you compute (25 + 15) * 2?"
    ]

    for question in test_questions:
        print(f"\n👤 Question: {question}")
        response = agent(question)
        print(f"🤖 Agent: {response}")
else:
    print("⚠️ No API key available")

## Part 3: Error Handling and Edge Cases

Good tools handle errors gracefully. Let's test how our calculator handles various error scenarios:

In [ ]:
model = create_working_model()

if model:
    agent = Agent(
        model=model,
        tools=[calculator],
        system_prompt="""You are a helpful assistant with a calculator tool.
        ALWAYS use the calculator tool for ANY mathematical expression or calculation request,
        even if you think it might have errors. The tool will handle errors and return
        appropriate error messages. After getting the tool's result, explain it to the user."""
    )

    # Test error cases
    error_test_cases = [
        "What is 10 divided by 0?",  # Division by zero
        "Calculate 2 +",  # Invalid syntax
        "What's the value of unknown_variable?",  # Unknown variable
        "Calculate hello + world",  # Invalid expression
    ]

    for test_case in error_test_cases:
        print(f"\n👤 Question: {test_case}")
        response = agent(test_case)
        print(f"🤖 Agent: {response}")
else:
    print("⚠️ No API key available")

## Part 4: Tool Selection Logic

The agent should be smart about **when** to use the calculator. It shouldn't use it for non-math questions!

Let's test with a mix of questions:

In [ ]:
model = create_working_model()

if model:
    agent = Agent(
        model=model,
        tools=[calculator],
        system_prompt="""You are a helpful assistant. Use the calculator tool when
        mathematical calculations are needed, but answer other questions normally
        without using tools."""
    )

    # Mix of math and non-math questions
    mixed_questions = [
        "What is 25 * 4?",  # Should use calculator
        "What is your name?",  # Should NOT use calculator
        "Tell me about Python programming",  # Should NOT use calculator
        "What's the square root of 144?",  # Should use calculator
        "What's the capital of France?",  # Should NOT use calculator
        "Calculate the area of a circle with radius 5 (use π ≈ 3.14159)",  # Should use calculator
    ]

    for question in mixed_questions:
        print(f"\n👤 Question: {question}")
        response = agent(question)
        print(f"🤖 Agent: {response}")
else:
    print("⚠️ No API key available")

## Part 5: Complex Mathematical Expressions

Let's push the calculator with more complex expressions:

In [ ]:
model = create_working_model()

if model:
    agent = Agent(
        model=model,
        tools=[calculator],
        system_prompt="""You are a mathematical assistant. Break down complex
        calculations step by step when helpful, and always verify your results."""
    )

    complex_questions = [
        "What is (15 + 25) * (30 - 10) / 4?",
        "Calculate 2**10 - 500",
        "What's the result of 100 * 0.15 + 50?",
        "Can you solve: 3 * (4 + 5) - 2 * 6?",
    ]

    for question in complex_questions:
        print(f"\n👤 Question: {question}")
        response = agent(question)
        print(f"🤖 Agent: {response}")
else:
    print("⚠️ No API key available")

## Part 6: Async Tool Usage

Tools work seamlessly with async agents. Let's run multiple calculations concurrently.

**Important:** When running concurrent queries, use separate agent instances to avoid conversation history contamination:

In [ ]:
model = create_working_model()

if model:
    # Define questions to ask
    questions = [
        "What is 123 + 456?",
        "Calculate 789 * 12",
        "What's 1000 / 25?",
    ]

    print("Running multiple calculations concurrently...")

    async def ask_question(question):
        """Helper function to ask a question with a fresh agent instance."""
        agent = Agent(
            model=model,
            tools=[calculator],
            system_prompt="You are a helpful calculator assistant."
        )
        return await agent.invoke_async(question)

    # Create concurrent tasks with separate agent instances
    tasks = [ask_question(q) for q in questions]

    # Wait for all results
    responses = await asyncio.gather(*tasks)

    # Display results
    for question, response in zip(questions, responses):
        print(f"\n👤 Q: {question}")
        print(f"🤖 A: {response}")
else:
    print("⚠️ No API key available")

## Experiments

Now it's your turn! Try these experiments:

### Exercises:
1. **Add more mathematical functions** - Modify the calculator to include log, exp, or trig functions
2. **Test with very large numbers** - Try calculations with numbers > 1 million
3. **Create a unit converter tool** - Follow the `@tool` pattern to convert units (meters to feet, etc.)
4. **Word problems** - Ask questions like "If I have 3 apples and buy 5 more, how many do I have?"
5. **Test floating point precision** - Try `0.1 + 0.2` and see what happens!

### Challenge:
Create a new tool that does something completely different (temperature conversion, currency exchange, etc.)

Use the cell below for your experiments:

In [ ]:
# Your experiments here!
import random

@tool
def fortune_teller(question: str) -> str:
    """
    Ask the fortune teller a question and receive a playful fortune.

    Parameters
    ----------
    question : str
        Any question you want to ask the fortune teller. 
        The content of the question does not affect the outcome— 
        the response is chosen randomly.

    Returns
    -------
    str
        A formatted string containing the question and a randomly 
        selected fortune response.

    Examples
    --------
    >>> fortune_teller("Will I get the promotion?")
    'Q: Will I get the promotion?\n🔮 Fortune: ✨ Great fortune awaits you soon.'

    >>> fortune_teller("Should I buy a lottery ticket?")
    'Q: Should I buy a lottery ticket?\n🔮 Fortune: ❌ No, it’s not the right time.'
    """
    fortunes = [
        "✨ Great fortune awaits you soon.",
        "⚠️ Be cautious—things may not go as planned.",
        "🌱 A new opportunity will grow from this.",
        "🎉 Yes! Celebrate, good news is on the way.",
        "❌ No, it’s not the right time.",
        "🤔 The answer is unclear, try again later.",
        "💡 Someone close to you holds the key.",
        "🌊 Go with the flow, it will work out naturally."
    ]
    
    response = random.choice(fortunes)
    return f"Q: {question}\n🔮 Fortune: {response}"

model = create_working_model()

if model:
    # Create agent with the calculator tool
    agent = Agent(
        model=model,
        tools=[fortune_teller],  # Register our calculator tool
        system_prompt="""You are a helpful assistant with fortune telling capabilities.
        When someone asks any question, use the fortune_teller tool to get results.
        Always explain your result clearly."""
    )

    # Test basic calculations
    test_questions = [
        "Will I get the promotion?",
        "I want to all in bitcoin",
        "Should i marry him?",
        "WIll i win TOTO?"
    ]

    for question in test_questions:
        print(f"\n👤 Question: {question}")
        response = agent(question)
        print(f"🤖 Agent: {response}")
else:
    print("⚠️ No API key available")

## ✅ Success Criteria

You've completed Lesson 2 if:

- ✅ Tool is correctly invoked for math questions
- ✅ Agent explains calculations clearly
- ✅ Handles errors gracefully (division by zero, invalid input)
- ✅ Agent knows when NOT to use the calculator
- ✅ Tool results are properly formatted in responses

## 💡 Key Concepts Learned

- **@tool decorator** - Creates reusable agent capabilities
- **Tool descriptions** - Help agents choose the right tool
- **Error handling** - Prevents agent crashes
- **Intelligent selection** - Agents decide when to use tools

## Next Steps

- **Lesson 3**: Multiple Tools - Combine weather, time, and calculation tools
- **Lesson 4**: Stateful Tools - Build tools with persistent data

Ready to continue? Open `lesson_03_multiple_tools.ipynb`!